In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd '/content/drive/MyDrive/Ali Sobhani Thesis/FFT'

/content/drive/.shortcut-targets-by-id/10ul3ZK_MC4nZuEgYotLTtUO0eEZB3gue/Ali Sobhani Thesis/FFT


In [ ]:
import numpy as np
from sklearn.experimental import enable_halving_search_cv  # noqa
from sklearn.model_selection import HalvingGridSearchCV, StratifiedKFold
from sklearn.feature_selection import SelectKBest, chi2, f_classif, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import fbeta_score
from scipy.stats import pearsonr

feature_selection = SelectKBest(score_func=f_classif)
feature_names = ['IP1_C1', 'IP1_C2', 'IP1_C3','IP1_C4','IN1_C1', 'IN1_C2', 'IN1_C3', 'IN1_C4',
                    'IP2_C1', 'IP2_C2', 'IP2_C3', 'IP2_C4', 'IN2_C1', 'IN2_C2', 'IN2_C3', 'IN2_C4',
                    'VP1_C1', 'VP1_C2', 'VP1_C3', 'VP1_C4', 'VN1_C1', 'VN1_C2', 'VN1_C3', 'VN1_C4',
                    'VP2_C1', 'VP2_C2', 'VP2_C3', 'VP2_C4', 'VN2_C1', 'VN2_C2', 'VN2_C3', 'VN2_C4']

X = np.load('X.npy')
Y = np.load('Y.npy')

knn = KNeighborsClassifier()

pipeline = Pipeline([
    ('feature_selection', feature_selection),
    ('knn', knn)
])

param_grid = {
    'feature_selection__k': list(range(1, X.shape[1] + 1)),
    'knn__n_neighbors': list(range(1, 21, 2)),
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan', 'minkowski']
}

outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=13)
inner_cv = StratifiedKFold(n_splits=2, shuffle=True, random_state=13)

grid_search = HalvingGridSearchCV(pipeline, param_grid, cv=inner_cv, scoring='accuracy')

best_scores = {'accuracy': [], 'f2': [], 'f1':[]}
best_params = []
all_selected_names = []

for train_idx, test_idx in outer_cv.split(X, Y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = Y[train_idx], Y[test_idx]

    grid_search.fit(X_train, y_train)
    best_params.append(grid_search.best_params_)

    #best_scores['accuracy'].append(grid_search.best_score_)

    y_pred = grid_search.predict(X_test)

    f2 = fbeta_score(y_test, y_pred, beta=2)
    ################################
    from sklearn.metrics import f1_score, accuracy_score
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    ################################
    best_scores['f2'].append(f2)
    best_scores['f1'].append(f1)
    best_scores['accuracy'].append(accuracy)

    selected_features = grid_search.best_estimator_.named_steps['feature_selection']
    selected_names = [feature_names[i] for i in selected_features.get_support(indices=True)]
    all_selected_names.append(selected_names)

average_best_scores = {scoring: np.mean(scores) for scoring, scores in best_scores.items()}
print(f"Average best scores: {average_best_scores}")

Streaming output truncated to the last 5000 lines.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:778: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 767, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 234, in __call__
    return self._score(
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 276, in _score
    y_pred = method_caller(estimator, "predict", X)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_scorer.py", line 73, in _cached_call
    return getattr(estimator, method)(*args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 481, in predict
    return self.st

Average best scores: {'accuracy': 1.0, 'f2': 1.0}


/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the test scores are non-finite: [0.5 0.5 0.5 ... 1.  1.  1. ]
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:952: UserWarning: One or more of the train scores are non-finite: [1. 1. 1. ... 1. 1. 1.]
  warnings.warn(


In [ ]:
import pandas as pd
import openpyxl
model = ['LR', 'LDA', 'SVM', 'KNN', 'XGBoost', 'RF']
FS = ['ANOVA', 'MI', 'Pearson', 'Chi2']
# Load the existing file
book = openpyxl.load_workbook('FFT_Results.xlsx')

# Prepare the data to be written
data_acc = average_best_scores['accuracy']
data_f2 = average_best_scores['f2']
data_f1 = average_best_scores['f1']

# Get the existing sheets
sheet_acc = book['ACC']
sheet_f2 = book['F2']
sheet_f1 = book['F1']

# Calculate the correct row and column numbers
row = model.index('KNN') + 2  # +2 because Excel index starts from 1 and row 1 contains headers
col = FS.index('ANOVA') + 2  # +2 because Excel index starts from 1 and column 1 contains headers

# Write to the ACC sheet
sheet_acc.cell(row=row, column=col, value=data_acc)

# Write to the F2 sheet
sheet_f2.cell(row=row, column=col, value=data_f2)

# Write to the F1 sheet
sheet_f1.cell(row=row, column=col, value=data_f1)

# Save and close the Excel file
book.save('FFT_Results.xlsx')

In [ ]:
import os
import pickle

# Specify the directory path
dir_path = '/content/drive/MyDrive/Ali Sobhani Thesis/FFT/ANOVA/KNN'

# Save best_params and all_selected_names to the directory
with open(os.path.join(dir_path, 'best_params.pkl'), 'wb') as f:
    pickle.dump(best_params, f)
with open(os.path.join(dir_path, 'all_selected_names.pkl'), 'wb') as f:
    pickle.dump(all_selected_names, f)